In [1]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
import statsmodels.api as sm

from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation_arcticle_1 import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ismailakrout/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ismailakrout/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

## Data PreProcessing

In [4]:
# On lit la base des speechs
# on peut la rendre plus petite pour faire des tests 
df = read_HouseOfCommons(keep_date=False, rd_lines=False, n=10000)

In [5]:
# on ne garde que Labor et Conservative
df = keep_parties(df, ['Lab', 'Con'])

## Data Processing

In [6]:
# On cleen les titres des speechs
df['agenda'] = df['agenda'].apply(clean, args=('unigram',))

In [7]:
# On construit las liste cleen des topics
list_stem_topics = process_list_BigTech_words(topics)

In [8]:
# On filtre le df et on garde uniquement les speechs sur les BigTech
df = keep_Bigtech_speeches(df, list_stem_topics)

In [9]:
df['text'] = df['text'].apply(clean, args=('bigram',))

In [10]:
# On fait le groupby() par speaker et party par la suite pour ne pas dépasser les 1 000 000 carc 
df = df.groupby(by=['Speaker', 'party']).sum().reset_index()

On va repérer les 100 mots (puis 500 quand on aura les speeches en entier) qui sont le plus utilisés par chaque parti, et noter leur fréquence d'apparition. 

In [11]:
df_freqs_Con = count_freqs(df, 'Con')
df_freqs_Lab = count_freqs(df, 'Lab')

On a tous les mots utilisés par le parti Conservateur sur les sujets définis, ainsi que leurs fréquences d'apparition. On va retenir les fréquences avec lesquelles chaques mots sont utilisés, par chaque parti, afin de déterminer le chi2 de chaque mot. On se restreint pour l'instant aux partis Con et Lab, fortement majoritaires. 

In [12]:
df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)

## feature selection

In [13]:
# On effectue le etst du Qi 2 
df_freqs = test_qi_2(df_freqs)

On a donc ici chaque mot utilisé par les parti sur le sujet délimité, avec le chi2 correspondant. On va se limiter aux 500 premiers mots :

In [14]:
df_freqs = keep_significatif_word(df_freqs, n=500)

## Data Processing Bis

In [15]:
list_of_words = selected_words(df_freqs)

In [16]:
df = construct_df_reg(df, df_freqs, list_of_words)

In [17]:
df = normalize(df, list_of_words)

In [18]:
df = party_str_to_dummy(df)

## Regressions linéaires (Modélisation)

In [19]:
# rajout des lignes slopes_coeff et intercept

df = regress_df(df, list_of_words)

In [20]:
df.to_csv('01. output/df_500_with_slope_coef_tierce.csv')

In [6]:
df = pd.read_csv('01. output/archives/df_500_with_slope_coef_bis.csv')
df

,Unnamed: 0,Speaker,party,text,party_bool,hon friend,right hon,friend member,hon member,hon gentleman,hon ladi,european union,thi countri,unit kingdom,human right,thank hon,give way,would like,absolut right,labour parti,thi hous,make sure,thi bill,northern ireland,stand order,previou govern,veri import,billion,thi govern,mr speaker,friend right,council europ,take place,side hous,mr deputi,deputi speaker,bill ha,friend know,royal mail,friend make,made hon,thi parliament,secretari state,member christchurch,code practic,wa veri,friend absolut,arm forc,borough council,import point,new claus,hous lord,thi debat,domest violenc,ha taken,friend minist,emerg worker,crime commission,member parliament,law enforc,ani way,assur hon,dog meat,reassur hon,thi point,veri clear,local author,mani year,last year,work hard,shadow minist,polic crime,friend ha,veri happi,import issu,member shipley,hope hon,thi place,congratul hon,pay tribut,across hous,member state,three year,lord amend,friend hi,deal thi,around world,member north,leav european,would say,tell hous,polic forc,road traffic,thi stage,great deal,thi issu,bill wa,right thing,everi singl,home offic,young carer,grate hon,thi matter,uk govern,anim welfar,devolv administr,let agent,one thing,select committe,whole hous,peopl come,everi day,look care,draft bill,veri long,shadow secretari,take view,remot gambl,made veri,help save,agre hon,member bill,car park,opposit member,privat sector,ha done,work ha,scottish govern,england wale,good point,scottish parliament,british peopl,labour member,come back,thi year,doe hon,work thi,thi problem,good thing,friend secretari,hous build,town centr,bill hous,uk economi,member thi,piec legisl,made clear,long time,mani peopl,would requir,coalit govern,free movement,last govern,bill would,privat member,thi import,make veri,pleasur follow,one reason,peopl want,rais thi,give hon,interest thi,devolv power,import thi,free trade,wa great,ministri defenc,nation parti,scottish nation,want see,point made,urg govern,stop search,econom plan,chang way,govern thi,past month,us leav,peopl move,address issu,chesham amersham,freedom express,commit offenc,look issu,border forc,bae system,hope hous,see whether,say hon,peopl vote,thi morn,way hon,thi case,part unit,work close,gentleman hi,oversea territori,ani chang,take part,hi intervent,want ensur,famili member,know mani,point wa,around countri,peopl thi,follow hon,make good,veri good,east somerset,royal assent,last parliament,includ hon,ha long,declar interest,servic personnel,none less,would put,one two,buri north,becaus wa,bill also,ha chang,citizen live,remain part,privat capit,section act,member birmingham,european commiss,statement right,movement peopl,friend think,air show,ensur continu,forward propos,left us,togeth make,union member,essex polic,defenc budget,befor wa,govern spend,made import,highcost credit,citi council,peopl abl,member thirsk,neighbourhood plan,west lothian,shipley philip,polic offic,philip davi,bill provid,govern support,make differ,ha said,work togeth,control order,gentleman ha,put record,valu money,due cours,need ensur,hous common,labour govern,make clear,health servic,claus bill,gentleman know,take forward,veri pleas,year wa,member buri,hope thi,agre thi,certif offic,know ha,know thi,bill make,nation health,thi subject,support bill,social care,busi rate,becaus know,opportun make,health social,give hous,get work,use power,take issu,befor thi,year govern,hi bill,welcom fact,everi time,think would,doe right,make decis,high street,issu rais,system wa,commend bill,sever time,polit fund,fair trade,entir agre,view thi,accredit person,park area,save account,tri achiev,wa support,believ right,asylum seeker,complet agre,peopl put,set price,ha home,way deal,agre would,patient choic,thirsk malton,thi group,merchant navi,urg hon,excel work,addit provis,power entri,lothian question,number plate,express opinion,thi claus,control notic,help ensur,obtain agreement,one anoth,wa problem,quickli possibl,borough coun